In [1]:
import csv
import math
from operator import mul
from itertools import product, islice
from collections import defaultdict
import numpy as np

In [58]:
def kmer_slice(kmer, k):
    """ >>> list(kmer_slice('ATCGA', 2))
        [('A', 'T'), ('T', 'C'), ('C', 'G'), ('G', 'A')]
    """
    itr = iter(kmer)
    res = tuple(islice(itr, k))
    if len(res) == k:
        yield res
    for b in itr:
        res = res[1:] + (b,)
        yield res

In [59]:
list(kmer_slice("ATGC", 1)), list(kmer_slice("ATGC", 2)),list(kmer_slice("ATGC", 3)), list(kmer_slice("ATGC", 4))

([('A',), ('T',), ('G',), ('C',)],
 [('A', 'T'), ('T', 'G'), ('G', 'C')],
 [('A', 'T', 'G'), ('T', 'G', 'C')],
 [('A', 'T', 'G', 'C')])

In [60]:
def kmer_count_bases(kmer):
    """
    Returns a dictionary with bases as keys and
    their count in the kmer as values.
    >> kmer_count_bases("AGCCAT")
    defaultdict(int, {'A': 2, 'G': 1, 'C': 2, 'T': 1})
    """
    kbases = defaultdict(int)
    for base in kmer:
        kbases[base] = kbases.get(base, 0) + 1
    return kbases

In [20]:
kmer_count_bases("ATGC"), kmer_count_bases(""), kmer_count_bases("AAA")

(defaultdict(int, {'A': 1, 'T': 1, 'G': 1, 'C': 1}),
 defaultdict(int, {}),
 defaultdict(int, {'A': 3}))

In [61]:
def get_trimers_N(kmer, alphabet):
    # XYZW
    trimers_n = []
    for letter in alphabet:
        # X + ACGT + Z = xNz
        kmer = kmer[0] + letter + kmer[2]
        trimers_n.append(kmer)
    return trimers_n

In [22]:
len(get_trimers_N("ATGC", 'ACGT')), get_trimers_N("ATGC"[1:], 'ACGT')

(4, ['TAC', 'TCC', 'TGC', 'TTC'])

In [23]:
get_trimers_N("ATG", 'ACGT')

['AAG', 'ACG', 'AGG', 'ATG']

In [62]:
def get_tetra_Ns(kmer, alphabet):
    # XYZW
    tetra_n = []
    for letter in alphabet:
        # X N ZW
        kmer0 = kmer[0] + letter + kmer[2:]
        # XY N W
        kmer1 = kmer[:2] + letter + kmer[-1]
        tetra_n.append(kmer0)
        tetra_n.append(kmer1)
    return tetra_n

In [5]:
len(get_tetra_Ns("ATGC", 'ACGT'))

8

In [63]:
def get_N1N2_kmer(kmer, alphabet):
    # XYZW
    nntetras = []
    for j in product(alphabet, repeat=2):
        # X +ACGT+ACGT+ W
        tetra = kmer[0] + "".join(j) + kmer[-1]
        nntetras.append(tetra)
    return nntetras

In [7]:
len(get_N1N2_kmer("CTAG", 'ACGT')), get_N1N2_kmer("CTAG", 'ACGT')

(16,
 ['CAAG',
  'CACG',
  'CAGG',
  'CATG',
  'CCAG',
  'CCCG',
  'CCGG',
  'CCTG',
  'CGAG',
  'CGCG',
  'CGGG',
  'CGTG',
  'CTAG',
  'CTCG',
  'CTGG',
  'CTTG'])

In [64]:
def get_kmer_sub_strs(kmer):
    pref = kmer[:-1]
    mid = kmer[1:-1]
    suf = kmer[1:]
    return pref, mid, suf

In [9]:
get_kmer_sub_strs("ATGC")

('ATG', 'TG', 'TGC')

In [10]:
def get_data_from_csv(filename):
    data = defaultdict(float)
    with open(filename, 'r') as fh:
        header = fh.readline()
        csv_data = csv.reader(fh)
        for row in csv_data:
            d1, d2 = row[0], float(row[1])
            print(d1)
            data[d1] = data.get(d1, 0.0) + d2
    return data

In [223]:
filename_lengths = "/media/paulo/Paulo/pauloscchlogl/Genome_kmers/Genomes_new/Bacteria/Pseudomonadota/Gammaproteobacteria/Gammaproteobacteria_chr_lengths.tsv"
lengths = defaultdict(int)
with open(filename_lengths, "r") as fh:
    for line in fh:
        lendata = line.strip().split("\t")
        lengths[lendata[0]] = lengths.get(lendata[0], 0) + int(lendata[1])

In [224]:
N = lengths["CP007470.1"]

In [17]:
filename = "/media/paulo/Paulo/pauloscchlogl/Genome_kmers/Counts/Archaea/Asgard_group/C_Heimdallarchaeota/GCA_020348965.1/CHR/CP070760.1_k1_8_fr.counts"

In [35]:
kmc = defaultdict(float)
with open(filename, "r") as fh:
    header = fh.readline()
    for line in fh:
        line = line.strip().split(",")
        kmer = line[0]
        if set(kmer).issubset(set("ACGT")):
            kmc[line[0]] = kmc.get(line[0], 0.0) + float(line[2])

In [36]:
kmc.keys() == "1"

False

In [37]:
kmc["1"]

0.0

In [38]:
def group_by_length(kmer_counts, k):
    selected =  defaultdict(int)
    for kmer, count in kmer_counts.items():
        if len(kmer) == k and set(kmer).issubset("ACGT"):
            selected[kmer] = selected.get(kmer, 0) + count
    return selected

In [39]:
bases = group_by_length(kmc, k=1)
bases

defaultdict(int,
            {'A': 0.30849133337917256,
             'C': 0.18385631752503756,
             'G': 0.18385631752503756,
             'T': 0.30849133337917256})

In [40]:
di_freqs = group_by_length(kmc, k=2)
di_freqs

defaultdict(int,
            {'AC': 0.046447371464584934,
             'CA': 0.06000901786113191,
             'AA': 0.10871972671381346,
             'CG': 0.02475602503074343,
             'GC': 0.029391699955544415,
             'CT': 0.05950235728664409,
             'TT': 0.10871972671381346,
             'TG': 0.06000901786113191,
             'GG': 0.03955849870039503,
             'GA': 0.06843086364556701,
             'AG': 0.05950235728664409,
             'AT': 0.093777834249431,
             'TA': 0.07127975997153321,
             'GT': 0.046447371464584934,
             'TC': 0.06843086364556701,
             'CC': 0.03955849870039503})

In [43]:
def kmer_count_bases(kmer):
    kbases = defaultdict(int)
    for base in kmer:
        kbases[base] = kbases.get(base, 0) + 1
    return kbases

In [44]:
kmer_count_bases("ATGC")

defaultdict(int, {'A': 1, 'T': 1, 'G': 1, 'C': 1})

In [45]:
def hexamers_second_order_markov(kmer_counts, kmer):
    """Second order markov for haxamers, 
    according to ELHAI, 2001:
    NTEM2(w1w2w3w4w5w6) = N(w1w2w3)N(w2w3w4)N(w3w4w5)N(w4w5w6) /
                           N(w2w3)N(w3w4)N(w4w5); 
    N = counts of the sub kmers in the hexamer.
    **** It works only with counts and not frequencies ****
    """
    # ACGTAC
    # ACG CGT GTA TAC
    trimers = ["".join(km) for km in kmer_slice(kmer, 3)] 
    # CG GT TA
    mid_dinucs = ["".join(km) for km in kmer_slice(kmer[1:5], 2)]
    tri_prod = np.prod([kmer_counts[km] for km in trimers])
    mi_prod = np.prod([kmer_counts[km] for km in mid_dinucs])
    return tri_prod / mi_prod

In [46]:
hexamers_second_order_markov(kmc, "GATATC") * sum()

0.00034943890138679147

In [283]:
hexamers_second_order_markov(kmc, "1")

1.0

In [47]:
def hexamers_third_order_markov(kmer_counts, kmer):
    """Third order markov for haxamers, 
    according to ELHAI, 2001:
    NTEM3(w1w2w3w4w5w6) = N(w1w2w3w4)N(w2w3w4w5)N(w3w4w5w6) / 
                                  N(w2w3w4)N(w3w4w5); 
    N = counts of the sub kmers in the hexamer.
    """
    # GATATC
    # ['GATA', 'ATAT', 'TATC']
    tetramers = ["".join(km) for km in kmer_slice(kmer, 4)] 
    # ['ATA', 'TAT']
    mid_tri = ["".join(km) for km in kmer_slice(kmer[1:5], 3)]
    tetra_prod = np.prod([kmer_counts[km] for km in tetramers])
    mi_prod = np.prod([kmer_counts[km] for km in mid_tri])
    return tetra_prod / mi_prod

In [248]:
hexamers_third_order_markov(kmc, "GATATC")

542.4596753964753

In [48]:
def dinucleotides_relative_abundance(dinuc_data, bases_freqs):
    """
    rel_ab <= 0.78 and rel_ab >= 1.23 as suitable benchmarks for 
    assessing whether a dinucleotide is signiﬁcantly over-or 
    under-represented in a DNA sequence.
    (Karlin and Cardon, 1994; Karlin and Ladunga, 1994)
    Give similar results with KARLIN & CARDON, 1994 data:
    TA: 0.79, AA: 1.22,GC:1.41, TT:1.22, but I counted only
    one strand
    """
    kmer_rel_ab = defaultdict(float)
    for kmer in dinuc_data.keys():
        prod = np.prod([bases_freqs[base] for base in kmer])
        rel_ab = dinuc_data[kmer] / prod
        kmer_rel_ab[kmer] = kmer_rel_ab.get(kmer, 0.0) + rel_ab
    return kmer_rel_ab

In [49]:
di_rel = dinucleotides_relative_abundance(di_freqs, bases)

In [51]:
{k:v for k,v in di_rel.items() if v <= 0.78 or v >= 1.23}

{'CG': 0.7323586213370193, 'TA': 0.7489973709006568}

In [52]:
{k:v for k,v in di_rel.items() if v >= 1.23}

{}

In [54]:
def trinucleotide_relative_abundance(kmer, kmer_freqs, base_freqs, alphabet):
    """
    Karlin
    g(XYZ) = (f*(XYZ)f*(X)f*(Y)f*(Z)/(f*(XY)f*(YZ)f*(XNZ)
    """
    km = kmer_freqs[kmer]
    b1 = base_freqs[kmer[0]]
    b2 = base_freqs[kmer[1]]
    b3 = base_freqs[kmer[2]]
    d1 = kmer_freqs[kmer[:2]]
    d2 = kmer_freqs[kmer[1:]]
    # [Count_of(XAZ) + Count_of(XCZ) + Count_of(XGZ) + Count_of(XTZ)] / 1846259
    tri_n = np.sum([kmer_freqs[t] for t in get_trimers_N(kmer, alphabet)])
    return (km * b1 * b2 * b3) / (d1 * d2 * tri_n)

In [65]:
trinucleotide_relative_abundance("CTC", kmc, bases, "ACGT")

0.9913813609832315

In [66]:
trinucleotide_relative_abundance("CTA", kmc, bases, "ACGT")

0.9199118416331437

In [256]:
trinucleotide_relative_abundance("CTT", kmf, N, haem_b_freqs, "ACGT")

1.113211367430552

In [67]:
def get_tetra_XYNW(kmer, alphabet):
    # XYZW
    tetra_n = []
    for letter in alphabet:
        kmern = kmer[:2] + letter + kmer[-1]
        tetra_n.append(kmern)
    return tetra_n

In [68]:
def get_tetra_XNZW(kmer, alphabet):
    # XYZW
    tetra_n = []
    for letter in alphabet:
        # X N ZW
        kmern = kmer[0] + letter + kmer[2:]
        tetra_n.append(kmern)
    return tetra_n

In [69]:
get_tetra_XNZW("CTAG", "ACGT")

['CAAG', 'CCAG', 'CGAG', 'CTAG']

In [70]:
get_tetra_XYNW("CTAG", "ACGT")

['CTAG', 'CTCG', 'CTGG', 'CTTG']

In [71]:
def tetranucleotide_relative_abundance(kmer, kmer_freqs, alphabet):
    """
    t*(XYZW) = (f*(XYZW)f*(XY)f*(XNZ)f*(XN1N2W)f*(YZ)f*(YNW)f*(ZW)) / 
               (f*(XYZ)f*(XYNW)f*(XNZW)f*(YZW)f*(X)f*(Y)f*(Z)f*(W),
    """
    XYZW = kmer_freqs[kmer]
    b1 = kmer_freqs[kmer[0]]
    b2 = kmer_freqs[kmer[1]]
    b3 = kmer_freqs[kmer[2]]
    b4 = kmer_freqs[kmer[3]]
    XY =  kmer_freqs[kmer[:2]]
    XNZ = np.sum([kmer_freqs[km] for km in get_trimers_N(kmer, alphabet)])
    XN1N2W = np.sum([kmer_freqs[km] for km in get_N1N2_kmer(kmer, alphabet)])
    YZ = kmer_freqs[kmer[1:3]]
    YNW = np.sum([kmer_freqs[km] for km in get_trimers_N(kmer[1:], alphabet)])
    ZW =  kmer_freqs[kmer[2:]]
    XYZ = kmer_freqs[kmer[1:4]]
    XYNW = np.sum([kmer_freqs[km] for km in get_tetra_XYNW(kmer, alphabet)])
    XNZW = np.sum([kmer_freqs[km] for km in get_tetra_XNZW(kmer, alphabet)])
    YZW = kmer_freqs[kmer[1:]]
    numerator = XYZW * XY * XNZ * XN1N2W * YZ * YNW * ZW
    denominator = XYZ * XYNW * XNZW * YZW * b1 * b2 * b3 * b4
    return numerator / denominator

In [262]:
tetranucleotide_relative_abundance("CTAG", kmf, "ACGT")

0.6559472952917303

In [263]:
tetranucleotide_relative_abundance("GGCC", kmf, "ACGT")

0.4734824164788884

In [72]:
def hexamers_fourth_order_markov(kmer_counts, kmer):
    """Fourth order markov for haxamers, 
    according to ELHAI, 2001:
    NTEM4(w1w2w3w4w5w6) = N(w1w2w3w4w5)N(w2w3w4w5w6) / 
                                  N(w2w3w4w5); 
    N = counts of the sub kmers in the hexamer.
    works only with counts
    """
        # GATATC
    # ['GATAT', 'ATATC']
    pentamers = ["".join(km) for km in kmer_slice(kmer, 5)]
    # ATAT
    mid = kmer_counts[kmer[1:5]]
    pentamers_prod = np.prod([kmer_counts[km] for km in pentamers])
    return pentamers_prod / mid

In [73]:
hexamers_fourth_order_markov(kmc, "GCTAAT"), kmc["GCTAAT"]

(0.0002852178786632616, 0.00029056144265499014)

In [267]:
1.981295216972807e-35

1.981295216972807e-35

In [268]:
def pemn(kmer_expected, kmer_observed):
    """
    according to ELHAI, 2001:
    ratio of N(w1...wL) / NTEMn
    """
    return kmer_observed / kmer_expected

In [278]:
haem_hex = group_by_length(kmc, k=6)
haem_hex_freqs = {k:v/tot for k,v in haem_di.items()}
sum(haem_hex_freqs.values())

0.999999458364184

In [75]:
di

NameError: name 'di' is not defined

In [74]:
def get_expected_kmer(kmer_counts, kmer_data, markov_func):
    expected = defaultdict(float)
    for kmer in kmer_data:
        exp_data = markov_func(kmer_counts, kmer)
        expected[kmer] = expected.get(kmer, 0.0) + exp_data
    return expected

In [281]:
hf_fourth = get_expected_kmer(kmc, haem_hex, hexamers_fourth_order_markov)

In [282]:
hf_fourth["1"]

0.0

In [273]:
fh_third = get_expected_kmer(kmc, haem_hex, hexamers_third_order_markov)

In [274]:
fh_third["1"]

0.0

In [179]:
def variance(kmer_counts, kmer_expected):
    """
    Calculates the variance from a list of strings of length k.

    Inputs:
        kmer_expect - a dictionary-like object mapping kmer to their calculated
                       expected values.

    Outputs:

        variance - a dictionary-like object mapping kmer to their calculated
                   expect variance.
    kmer_var <- kmer_exp * (m - p) * (m - s) / (m^2)
    Because the model for the count is the sum of N almost independent observations,
    each with probability P(W), it can be well modeled as a binomial distribution,
    with varianceThe variance is calculated as:
    E(C(W)) * (1 - E(C(W))/N)
    """
    variance = defaultdict(float)
    for kmer in kmer_expected:
        p, m, s = get_kmer_sub_strs(kmer)
        pref = kmer_counts[p]
        mid = kmer_counts[m]
        suf = kmer_counts[s]
        ex_val = kmer_expected[kmer]
        if ex_val == 0:
            variance[kmer] = variance.get(kmer, 0.0)
        else:
            var = (ex_val * (mid - pref) * (mid - suf)) / (mid**2)
            variance[kmer] = variance.get(kmer, 0.0) + var
    return variance

In [180]:
hf_f_var = variance(kmc, hf_fourth)

In [181]:
hf_t_var = variance(kmc, fh_third)

In [182]:
def get_standard_deviation(variance_data):
    """
    Calculates the standard deviation from the kmers expected values.

    Inputs:
        variance - a dictionary-like object mapping kmer to their calculated
                   expected variance.

    Outputs:

        std - a dictionary-like object mapping kmer to their calculated
                   expected std.

    The variance is calculated as:
    sigma(W) = sqrt(variance))
    """
    # initialize the container
    std = defaultdict(float)
    # iterates through the kmer keys
    for kmer in variance_data:
        # deals with zero error division
        if variance_data[kmer] == 0.0:
            std[kmer] = std.get(kmer, 0.0)
        else:
            # calculates the standard deviation and add
            # the kmer and the standard deviation values to the container
            sd = math.sqrt(variance_data[kmer])
            std[kmer] = std.get(kmer, 0.0) + sd
    return std

In [183]:
hf_f_std = get_standard_deviation(hf_f_var)

In [184]:
hf_t_std = get_standard_deviation(hf_t_var)

In [185]:
def get_z_scores(kmer_data, expected_kmers, std):
    """Calculates the z_score of all palindromes.
    Input:
    palindrome_lst = list of palindromes (str)
    counts = dictionary of kmer counts
    expected = dictionary of kmer expected values
    length_sequence = length of sequence (int)
    Output:
    z_score dictionary where key are palindromes and values are the calculated z_score (float)
    The z_scores are calculated as:
        Z(W) = (C(W)) - E(C(W)) / sigma(W)
    And sigma as:
        sigma(W) = sqrt(E(C(W))) * (1 - E(C(W)/N))
    """
    z_score = defaultdict(float)
    for kmer in kmer_data:
        if expected_kmers[kmer] == 0.0 or std[kmer] == 0.0:
            z_score[kmer] = 0.0
        else:
            sigma = std[kmer]
            obs = kmer_data[kmer]
            exp = expected_kmers[kmer]
            z = round((obs - exp) / sigma, 5)
            z_score[kmer] = z_score.get(kmer, 0.0) + z
    return z_score

In [186]:
hf6_f_z = get_z_scores(haem_hex, hf_fourth, hf_f_std)

In [188]:
hf6_f_z = get_z_scores(haem_hex, fh_third, hf_t_std)

In [189]:
hf6_f_z

defaultdict(float,
            {'AACGTC': -3.35816,
             'ACGTCC': 6.68693,
             'CGTCCG': 1.34779,
             'GTCCGT': -1.5596,
             'TCCGTC': 3.6039,
             'CCGTCG': -2.06674,
             'CGTCGT': 5.62969,
             'GTCGTG': 4.52717,
             'TCGTGG': -2.60932,
             'CGTGGA': -2.58551,
             'GTGGAG': 3.06007,
             'TGGAGA': 4.62741,
             'GGAGAG': 5.42505,
             'GAGAGG': 3.11755,
             'AGAGGG': 8.7046,
             'GAGGGA': 3.03838,
             'AGGGAA': 0.62375,
             'GGGAAA': 5.5998,
             'GGAAAC': -3.29878,
             'GAAACA': -6.98036,
             'AAACAA': -4.87647,
             'AACAAC': 5.67071,
             'ACAACC': -0.81788,
             'CAACCC': -1.68866,
             'AACCCA': 4.50953,
             'ACCCAG': -0.29476,
             'CCCAGA': -3.3138,
             'CCAGAC': -0.71241,
             'CAGACC': -1.63824,
             'AGACCG': -2.51696,
           

In [205]:
del hf6_f_z["1"]

In [207]:
hf6_f_z["1"]

0.0

In [190]:
import pandas as pd

In [206]:
pd.DataFrame(hf6_f_z.items()).rename(columns={0:" kmer", 1: "z-score"})

,kmer,z-score
0,AACGTC,-3.35816
1,ACGTCC,6.68693
2,CGTCCG,1.34779
3,GTCCGT,-1.55960
4,TCCGTC,3.60390
...,...,...
4092,GGGACC,-6.30713
4093,CGGACC,-6.56860
4094,GTCGAC,-10.46749
4095,GGCCGG,-1.35403


In [201]:
dfHi = pd.DataFrame(hf6_f_z.items()).rename(columns={0:" kmer", 1: "z-score"})

In [202]:
dfHi

,kmer,z-score
0,AACGTC,-3.35816
1,ACGTCC,6.68693
2,CGTCCG,1.34779
3,GTCCGT,-1.55960
4,TCCGTC,3.60390
...,...,...
4092,GGGACC,-6.30713
4093,CGGACC,-6.56860
4094,GTCGAC,-10.46749
4095,GGCCGG,-1.35403


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
x = list(range(1, len(hf6_f_z) + 1))
y = list(hf6_f_z.values())

In [ ]:
x

In [ ]:
ax = plt.subplot(111)
ax.plot(x, y)

plt.axhline(
    np.mean(y),
    color='r',
    linestyle='-',
    linewidth=2,
    label='Mean'
)
plt.show()

In [ ]:
{k:v for k,v in di_rel_hif.items() if v <= 0.78}

In [ ]:
{k:v for k,v in di_rel_hif.items() if v >= 1.23}

In [ ]:
x, y = di_rel_hif.keys(), np.array([v for v in di_rel_hif.values()])

In [ ]:
y

In [ ]:
ax = plt.subplot(111)
ax.plot(x, y)

plt.axhline(
    np.mean(y),
    color='r',
    linestyle='-',
    linewidth=2,
    label='Mean'
)
plt.show()

In [ ]:
def distance_dinuc_rel_ab(data1, data2):
    """
    Measure of difference between two sequences f and g (from different 
    organisms or from different regions of the same genome) is the average 
    absolute dinucleotide relative abundance difference calculated as:
    d(f,g) = 1/16sum(abs(p*XYf - p*XYg)). KARLIN AND MRAZEK, 1997.
    """
    return 1/16 * np.sum(abs(data1 - data2))

In [ ]:
573663.0+358056.0+346429.0+568111.0

In [ ]:
kmer_counts = {'A': 573663.0,
        'C': 358056.0,
        'G': 346429.0,
        'T': 568111.0,
        'AA': 221810.0,
        'AC': 95281.0,
        'CG': 73682.0,
        'GT': 89763.0,
        'TC': 96984.0,
        'CC': 69362.0,
        'TG': 119094.0,
        'GG': 65650.0,
        'GA': 94587.0,
        'AG': 88003.0,
        'CA': 124546.0,
        'GC': 96429.0,
        'CT': 90466.0,
        'TA': 132719.0,
        'AT': 168569.0,
        'TT': 219313.0,
        'AAC': 35495.0,
        'ACG': 21707.0,
        'CGT': 20667.0,
        'GTC': 11216.0,
        'TCC': 16456.0,
        'CCG': 13302.0,
        'TCG': 18894.0,
        'GTG': 22299.0,
        'TGG': 26651.0,
        'GGA': 16021.0,
        'GAG': 12324.0,
        'AGA': 25750.0,
        'AGG': 15668.0,
        'GGG': 10736.0,
        'GAA': 38028.0,
        'AAA': 83725.0,
        'ACA': 28247.0,
        'CAA': 48554.0,
        'ACC': 22123.0,
        'CCC': 11691.0,
        'CCA': 27744.0,
        'CAG': 20717.0,
        'GAC': 11574.0,
        'CGC': 21067.0,
        'GCC': 19092.0,
        'AGC': 24429.0,
        'GCT': 24491.0,
        'CTA': 19544.0,
        'TAA': 51502.0,
        'AAG': 36233.0,
        'GGT': 20433.0,
        'AGT': 22156.0,
        'TCT': 26146.0,
        'TAT': 38683.0,
        'ATA': 38687.0,
        'ATT': 66131.0,
        'TTA': 51706.0,
        'CGA': 19353.0,
        'GGC': 18460.0,
        'CTT': 37165.0,
        'TAG': 18729.0,
        'GAT': 32661.0,
        'ATG': 30067.0,
        'TGT': 26507.0,
        'GTT': 33466.0,
        'TTG': 46281.0,
        'GCA': 33067.0,
        'CAT': 30868.0,
        'ATC': 33684.0,
        'TCA': 35488.0,
        'TTT': 82551.0,
        'GCG': 19779.0,
        'GTA': 22782.0,
        'AAT': 66357.0,
        'CTC': 13309.0,
        'CAC': 24407.0,
        'ACT': 23204.0,
        'CGG': 12595.0,
        'CCT': 16625.0,
        'CTG': 20447.0,
        'TGC': 32473.0,
        'TGA': 33463.0,
        'TAC': 23805.0,
        'TTC': 38775.0,
        'AACG': 8029.0,
        'ACGT': 5659.0,
        'CGTC': 2736.0,
        'GTCC': 1617.0,
        'TCCG': 2983.0,
        'CCGT': 3964.0,
        'GTCG': 2839.0,
        'TCGT': 5665.0,
        'CGTG': 4975.0,
        'GTGG': 5540.0,
        'TGGA': 6424.0,
        'GGAG': 1890.0,
        'GAGA': 3589.0,
        'AGAG': 3258.0,
        'GAGG': 2010.0,
        'AGGG': 2498.0,
        'GGGA': 2972.0,
        'GGAA': 6673.0,
        'GAAA': 15039.0,
        'AAAC': 13453.0,
        'AACA': 11634.0,
        'ACAA': 11982.0,
        'CAAC': 8723.0,
        'AACC': 7761.0,
        'ACCC': 3044.0,
        'CCCA': 4626.0,
        'CCAG': 3987.0,
        'CAGA': 5824.0,
        'AGAC': 2464.0,
        'GACC': 2369.0,
        'ACCG': 5069.0,
        'CCGC': 5359.0,
        'CGCC': 5883.0,
        'GCCA': 7937.0,
        'CAGC': 6120.0,
        'AGCT': 5683.0,
        'GCTA': 5267.0,
        'CTAA': 8701.0,
        'TAAG': 6339.0,
        'AAGG': 6323.0,
        'AGGT': 4809.0,
        'GGTC': 2294.0,
        'TCCC': 3126.0,
        'CCCC': 2287.0,
        'CCAA': 10230.0,
        'CAAG': 9012.0,
        'AAGT': 9893.0,
        'AGTC': 2678.0,
        'GTCT': 2409.0,
        'TCTA': 5957.0,
        'CTAT': 5302.0,
        'TATA': 6131.0,
        'ATAT': 11417.0,
        'TATT': 17305.0,
        'ATTA': 16607.0,
        'TTAA': 17067.0,
        'AGTG': 6111.0,
        'TGGG': 4380.0,
        'ACGA': 6142.0,
        'CGAA': 6247.0,
        'GAAG': 6973.0,
        'AGGC': 4575.0,
        'GGCT': 5336.0,
        'GCTT': 9618.0,
        'CTTA': 6705.0,
        'TTAG': 8462.0,
        'TAGA': 5741.0,
        'GACA': 3547.0,
        'ACAG': 4870.0,
        'CTAG': 1971.0,
        'TAGG': 3335.0,
        'AGGA': 3786.0,
        'GGAT': 5715.0,
        'GATG': 7666.0,
        'ATGT': 6486.0,
        'TGTT': 10551.0,
        'GTTG': 7940.0,
        'TTGG': 9927.0,
        'TGGC': 7700.0,
        'AGAA': 11805.0,
        'AAGC': 9421.0,
        'AGCA': 8793.0,
        'GCAG': 5898.0,
        'AGCC': 5320.0,
        'CCAT': 7462.0,
        'CATC': 8025.0,
        'ATCA': 12454.0,
        'TCAT': 9047.0,
        'CATT': 13859.0,
        'ATTT': 25080.0,
        'TTTA': 21278.0,
        'TAAA': 21339.0,
        'AAAG': 13909.0,
        'AAGA': 10596.0,
        'AGCG': 4633.0,
        'GCGT': 5379.0,
        'CGTA': 5179.0,
        'GTAA': 9750.0,
        'TAAT': 16472.0,
        'AATA': 17301.0,
        'ATAG': 4867.0,
        'TAGC': 5262.0,
        'GCTC': 3857.0,
        'CTCA': 4628.0,
        'TCAC': 6655.0,
        'CACT': 6632.0,
        'ACTA': 4791.0,
        'TAGT': 4391.0,
        'TCGA': 3625.0,
        'CGAG': 3088.0,
        'GAGT': 3099.0,
        'TCGG': 3133.0,
        'CGGC': 3242.0,
        'GGCC': 989.0,
        'GCCT': 4632.0,
        'CCTG': 3993.0,
        'CTGC': 5945.0,
        'TGCG': 6893.0,
        'GCGC': 3247.0,
        'CGCG': 2749.0,
        'GCGG': 4956.0,
        'CGGA': 2839.0,
        'AGAT': 8223.0,
        'TGTA': 6300.0,
        'TAAC': 7352.0,
        'ACGG': 3916.0,
        'CGGG': 1810.0,
        'GGGG': 2048.0,
        'GGGC': 2943.0,
        'GCTG': 5748.0,
        'CTGA': 5687.0,
        'TGAA': 13303.0,
        'AAAT': 25321.0,
        'GCAC': 6988.0,
        'CACC': 5707.0,
        'CCGA': 3389.0,
        'GGCA': 6631.0,
        'GCAT': 7663.0,
        'TCAG': 5962.0,
        'CAGG': 4000.0,
        'GGCG': 5504.0,
        'GTAT': 6087.0,
        'TATC': 8535.0,
        'ATAC': 6419.0,
        'TACG': 5414.0,
        'ACGC': 5990.0,
        'CCTT': 6720.0,
        'TTAC': 10300.0,
        'CGAT': 7017.0,
        'GATT': 11714.0,
        'ACAT': 6696.0,
        'CATG': 2060.0,
        'ATGG': 7185.0,
        'GAAT': 10050.0,
        'AATG': 13629.0,
        'GCGA': 6197.0,
        'ATGA': 8859.0,
        'GGTT': 7476.0,
        'TTGT': 10910.0,
        'GTTC': 5648.0,
        'TTCA': 14055.0,
        'TCAA': 13824.0,
        'GCAA': 12518.0,
        'CCCG': 1961.0,
        'GAAC': 5966.0,
        'CGTT': 7777.0,
        'TTGA': 12969.0,
        'TGAG': 4088.0,
        'GGGT': 2773.0,
        'GGTA': 5598.0,
        'TTTT': 30456.0,
        'TTTG': 15526.0,
        'AGTA': 5705.0,
        'TTAT': 15877.0,
        'GTGA': 5948.0,
        'TGAT': 11706.0,
        'GATA': 8331.0,
        'CTGT': 4816.0,
        'GTAG': 3429.0,
        'GAGC': 3626.0,
        'TGTG': 6148.0,
        'GTGT': 4295.0,
        'GTTA': 7116.0,
        'AATC': 12174.0,
        'ATCG': 6848.0,
        'CTGG': 3999.0,
        'GGAC': 1743.0,
        'GACG': 2865.0,
        'CACA': 6669.0,
        'GTGC': 6516.0,
        'ATGC': 7537.0,
        'TGCT': 8450.0,
        'TGAC': 4366.0,
        'CATA': 6924.0,
        'ATAA': 15984.0,
        'AAAA': 31042.0,
        'GGTG': 5065.0,
        'TTCG': 6032.0,
        'TCGC': 6471.0,
        'GCCG': 3289.0,
        'CCGG': 590.0,
        'ACCA': 8596.0,
        'TTCC': 6910.0,
        'TCCT': 3762.0,
        'TGTC': 3508.0,
        'TCCA': 6585.0,
        'CTCC': 1955.0,
        'CCTA': 3529.0,
        'ATTC': 10435.0,
        'GTAC': 3516.0,
        'TACT': 5708.0,
        'ACTT': 10293.0,
        'CTTG': 8806.0,
        'TGGT': 8147.0,
        'CGCA': 7413.0,
        'ACTG': 4827.0,
        'TATG': 6712.0,
        'GTTT': 12762.0,
        'AGTT': 7662.0,
        'CAAA': 16305.0,
        'ATCC': 5974.0,
        'GACT': 2793.0,
        'CTTC': 7401.0,
        'ACAC': 4699.0,
        'CTCT': 3551.0,
        'CTAC': 3570.0,
        'AACT': 8071.0,
        'TACC': 6286.0,
        'CCAC': 6065.0,
        'CTTT': 14253.0,
        'CGAC': 3001.0,
        'GTCA': 4351.0,
        'ACTC': 3293.0,
        'CGCT': 5022.0,
        'CTCG': 3175.0,
        'ATTG': 14009.0,
        'CCCT': 2817.0,
        'CGGT': 4704.0,
        'TGCA': 10230.0,
        'TCTG': 5879.0,
        'CACG': 5399.0,
        'TGCC': 6900.0,
        'GCCC': 3234.0,
        'AATT': 23253.0,
        'ACCT': 5414.0,
        'GATC': 4950.0,
        'CAGT': 4773.0,
        'TCTC': 3776.0,
        'TTCT': 11778.0,
        'CCTC': 2383.0,
        'TTGC': 12475.0,
        'CAAT': 14514.0,
        'ATCT': 8408.0,
        'TACA': 6397.0,
        'TCTT': 10534.0,
        'TTTC': 15291.0}

In [ ]:
def get_trimers_N(kmer, alphabet):
        # get a list of all timers XYZW
        trimers_n = []
        for letter in alphabet:
            # X + {ACGT} + Z = xNz
            kmer = kmer[0] + letter + kmer[2]
            trimers_n.append(kmer)
        return trimers_n
    
    
def get_tetra_XNZW(kmer, alphabet):
        # get a list of all tetramers XYZW
        tetra_n = []
        for letter in alphabet:
            # X N ZW
            kmern = kmer[0] + letter + kmer[2:]
            tetra_n.append(kmern)
        return tetra_n
    
    
def get_tetra_XYNW(kmer, alphabet):
        # get a list of all tetramers XYZW
        tetra_n = []
        for letter in alphabet:
            kmern = kmer[:2] + letter + kmer[-1]
            tetra_n.append(kmern)
        return tetra_n
    
    
def get_N1N2_kmer(kmer, alphabet):
        # get a list of all tetramers XN1N2W
        nntetras = []
        for j in product(alphabet, repeat=2):
            # X +ACGT+ACGT+ W
            tetra = kmer[0] + "".join(j) + kmer[-1]
            nntetras.append(tetra)
        return nntetras
    
    
def trinucleotide_relative_abundance(kmer, kmer_freqs, alphabet):
        """
        Karlin
        g(XYZ) = (f*(XYZ)f*(X)f*(Y)f*(Z)/(f*(XY)f*(YZ)f*(XNZ)
        """
        bases_counts = {k:v for k,v in kmer_freqs.items() if len(k) == 1}
        N = sum(bases_counts.values())
        km = kmer_freqs[kmer]
        # A C G T counts or frequencies multiplication
        bases = np.sum([bases_counts[b] for b in kmer])
        # multiplication of all dimers counts/freqs
        di = np.sum([kmer_freqs[kmer[:2]], kmer_freqs[kmer[1:]]]) / N
        # multiplicatiion of all XNZ trimers counts/freqs
        tri_n = np.sum([kmer_freqs[t] for t in get_trimers_N(kmer, alphabet)]) / N
        return (km * bases) / (di * tri_n)
    
    

In [ ]:
alphabet = "ACGT"
kmer = "GAC"
bases = {'A': 573663.0, 'C': 358056.0, 'G': 346429.0, 'T': 568111.0}
total = sum(bases.values())
kmer_freqs = {k:v/total for k,v in kmer_counts.items()}
if len(kmer) == 3:
    trimers_rel_ab_freqs = trinucleotide_relative_abundance(kmer, kmer_freqs, alphabet)
    print(f"Trimers relative abundance calculated with frequencies: {trimers_rel_ab_freqs}\n")
else:
    tetra_rel_ab_freqs = tetranucleotide_relative_abundance(kmer, kmer_freqs, alphabet)
    print(f"Tetranucleotides relative abundance calculated with frequencies: {tetra_rel_ab_freqs}\n")


In [ ]:
for km in [k for k in kmf if len(k) == 3]:
    tri_rel = trinucleotide_relative_abundance(km, kmer_freqs, alphabet)
    if tri_rel <= 0.78:
        print(f"Under-represented trimers - {km}: {tri_rel}")
    elif tri_rel >= 1.23:
        print(f"Over-represented trimer - {km}: {tri_rel}")              

In [ ]:
def tetranucleotide_relative_abundance(kmer, kmer_freqs, N, alphabet):
        """
        t*(XYZW) = (f*(XYZW)f*(XY)f*(XNZ)f*(XN1N2W)f*(YZ)f*(YNW)f*(ZW)) / 
                   (f*(XYZ)f*(XYNW)f*(XNZW)f*(YZW)f*(X)f*(Y)f*(Z)f*(W),
        """
        bases_counts = {k:v/N for k,v in kmer_freqs.items() if len(k) == 1}
        print(bases)
        XYZW = kmer_freqs[kmer]
        print(kmer, XYZW)
        k1 = bases_counts[kmer[0]]
        print(k1)
        k2 = bases_counts[kmer[1]]
        print(k2)
        k3 = bases_counts[kmer[2]]
        print(k2)
        k4 = bases_counts[kmer[3]]
        print(k2)
        sum_ks = (k1+k2+k3+k4) / N
        print(sum_ks)
#         bases = np.prod([kmer_freqs[b] for b in kmer])
#         XY =  kmer_freqs[kmer[:2]]
#         # multiplicatiion of all XNZ trimers counts/freqs
#         XNZ = np.sum([kmer_freqs[km] for km in get_trimers_N(kmer, alphabet)])  / N
#         # multiplicatiion of all XN1N2W tetramers counts/freqs
#         XN1N2W = np.sum([kmer_freqs[km] for km in get_N1N2_kmer(kmer, alphabet)]) / N
#         YZ = kmer_freqs[kmer[1:3]]
#         # multiplicatiion of all YNW trimers counts/freqs
#         YNW = np.sum([kmer_freqs[km] for km in get_trimers_N(kmer[1:], alphabet)]) / N
#         ZW =  kmer_freqs[kmer[2:]]
#         XYZ = kmer_freqs[kmer[1:4]]
#         # multiplicatiion of all XYNW tetramers counts/freqs
#         XYNW = np.sum([kmer_freqs[km] for km in get_tetra_XYNW(kmer, alphabet)]) / N
#         # multiplicatiion of all XNZW tetramers counts/freqs
#         XNZW = np.sum([kmer_freqs[km] for km in get_tetra_XNZW(kmer, alphabet)]) / N
#         YZW = kmer_freqs[kmer[1:]]
#         return (XYZW * XY * XNZ * XN1N2W * YZ * YNW * ZW) / (XYZ * XYNW * XNZW * YZW * sum_ks)

In [ ]:
tetranucleotide_relative_abundance("TGCA", kmer_freqs, N, alphabet)

In [ ]:
for km in [k for k in kmer_freqs if len(k) == 4]:
    tetra_rel = tetranucleotide_relative_abundance(km, kmer_freqs, N, alphabet)
    if tetra_rel <= 0.78:
        print(f"Under-represented tetramer - {km}: {tetra_rel}")
    elif tetra_rel >= 1.23:
        print(f"Over-represented tetramer - {km}: {tetra_rel}")
    else:
        print(f"Normaly distributed tetramer - {km}:{tetra_rel}")

In [ ]:
kmer_freqs